In [30]:
import pandas as pd

In [31]:
df=pd.read_csv("datasets/tr_yemekler_arttirilmis.csv")
df.head()

,foodname,materials
0,karnıyarık,"patlcan,biber,soan,domates"
1,mantı,"un,yumurta,et,soan,yourt,sarmsak"
2,pizza,"hamur,peynir,domates,pepperoni"
3,salata,"marul,domates,salatalk,zeytinya"
4,çorba,"tavuk,sebze,makarna"


In [32]:
df["foodname"].value_counts()

foodname
karnıyarık         6
mantı              6
pizza              6
salata             6
çorba              6
                  ..
gyro               6
spanakopita        6
baklava (yunan)    6
dolmades           6
pastitsio          6
Name: count, Length: 150, dtype: int64

In [33]:
df.head()

,foodname,materials
0,karnıyarık,"patlcan,biber,soan,domates"
1,mantı,"un,yumurta,et,soan,yourt,sarmsak"
2,pizza,"hamur,peynir,domates,pepperoni"
3,salata,"marul,domates,salatalk,zeytinya"
4,çorba,"tavuk,sebze,makarna"


In [34]:
df.isnull().sum()

foodname     0
materials    0
dtype: int64

In [35]:
import re
def clean_materials(text):
    text = text.lower()  
    text = re.sub(r'\(.*?\)', ' ', text)  
    text = re.sub(r'[^a-zA-Z0-9\s,]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

In [36]:
df.head()

,foodname,materials
0,karnıyarık,"patlcan,biber,soan,domates"
1,mantı,"un,yumurta,et,soan,yourt,sarmsak"
2,pizza,"hamur,peynir,domates,pepperoni"
3,salata,"marul,domates,salatalk,zeytinya"
4,çorba,"tavuk,sebze,makarna"


In [37]:
df.to_csv("datasets/tr_yemekler_arttirilmis_temiz.csv",index=False, encoding="utf-8")

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

X=df["materials"]
y=df["foodname"]

vectorizer=TfidfVectorizer(max_features=1000,ngram_range=(1,2), stop_words='english')
X_vec=vectorizer.fit_transform(X)

In [39]:
import warnings
warnings.filterwarnings("ignore")

X_train,X_test,y_train,y_test=train_test_split(X_vec,y,test_size=0.2,random_state=42)

models={
    "Logistic Regression":LogisticRegression(max_iter=1000),
    "Linear SVC":LinearSVC(),
    "Random Forest Classifier":RandomForestClassifier(n_estimators=200, max_depth=25, random_state=42, n_jobs=-1),
    "MultinomialNB":MultinomialNB()
}

results=[]

for name, model in models.items():
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"\n{name} Modeli")
    print("-" * 40)
    print(f"Eğitim doğruluğu: {train_acc:.4f}")
    print(f"Test doğruluğu:   {test_acc:.4f}")
    print(f"Fark: {abs(train_acc - test_acc):.4f}")


Logistic Regression Modeli
----------------------------------------
Eğitim doğruluğu: 0.9861
Test doğruluğu:   0.8889
Fark: 0.0972

Linear SVC Modeli
----------------------------------------
Eğitim doğruluğu: 0.9986
Test doğruluğu:   0.9389
Fark: 0.0597

Random Forest Classifier Modeli
----------------------------------------
Eğitim doğruluğu: 0.9944
Test doğruluğu:   0.8556
Fark: 0.1389

MultinomialNB Modeli
----------------------------------------
Eğitim doğruluğu: 0.9722
Test doğruluğu:   0.8278
Fark: 0.1444


In [40]:
model_linear_svc=LinearSVC()
model_linear_svc.fit(X_train,y_train)
y_pred=model_linear_svc.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print(acc)

0.9388888888888889


In [41]:
samples = [
    "patlcan,biber,soğan,domates",
    "biber,tavuk,makarna",
    "yufka,peynir,yumurta",
    "pirinç, kıyma, baharat"
]

# TF-IDF dönüşümü
X_new = vectorizer.transform(samples)

# Model tahmini
predictions = model_linear_svc.predict(X_new)

# Sonuçları yazdır
print("\n🔹 Örnek Malzeme Tahminleri:")
for ingr, pred in zip(samples, predictions):
    print(f"Malzemeler: {ingr}")
    print(f"➡️ Tahmin Edilen Yemek: {pred}\n")


🔹 Örnek Malzeme Tahminleri:
Malzemeler: patlcan,biber,soğan,domates
➡️ Tahmin Edilen Yemek: karnıyarık

Malzemeler: biber,tavuk,makarna
➡️ Tahmin Edilen Yemek: çorba

Malzemeler: yufka,peynir,yumurta
➡️ Tahmin Edilen Yemek: börek

Malzemeler: pirinç, kıyma, baharat
➡️ Tahmin Edilen Yemek: kebap



In [42]:
import joblib

joblib.dump(model_linear_svc,"models/model3_linear_svc.pkl")
joblib.dump(vectorizer,"models/vectorizer_3.pkl")
print("Dışa Aktarıldı")

Dışa Aktarıldı
